In [ ]:
#test the geo distance matrix, added in the function files
#weighted cluster method
library('data.table')
library('ggplot2')
library('ggmap')
library('dbscan')
library('dplyr')
library('xts')
#smu
#funPath='D:/share/Git/Rprojects/ECA/function.R'

funPath='D:/Git/Rprojects/ECA/function.R'
source(funPath)

In [ ]:
#convert the csv document to esri shp documents
#must have a lon and lat column as longitude and latitude
#infile="D:/Git/data/terminals.csv"
data2shp<-function(MyData,filepath){ 
    library(rgdal)
    library('sp')
    coordinates(MyData)<-~lon+lat # whatever the equivalent is in your 
    writeOGR(MyData, filepath, "layer name", driver = "ESRI Shapefile")  
}

# functions for a ship

In [ ]:
#get stay points based data with status=5 and sog=0, mainly berth area
#also work for database with more then one mmsi
getStayPoint<-function(dt,eps=3600*2,minp=5){
    setkey(dt,mmsi,time)
    temp3=dt[,.N,list(mmsi)]
    nr=nrow(temp3)#how many ships
    r0=data.table(mmsi=0,stayid=0,startpid=0,endpid=0,duration=0,lon=0,lat=0)[mmsi<0]
    for (i in seq(1,nr)){
        r=temp3[i]
        cship=dt[mmsi==r$mmsi]
        cship2 <- as.matrix(cship[,list(time)])#get time series
        cl2 <- dbscan(cship2, eps = eps, minPts =minp)
        cship3=data.table(cbind(cship,stayid=cl2$cluster));
        
        temp=cship3[,list(startpid=first(.SD)$pid,endpid=last(.SD)$pid,duration=(last(.SD)$time-first(.SD)$time),lon=mean(.SD$lon),lat=mean(.SD$lat)),list(mmsi,stayid)]
        
        r0=rbind(r0,temp)      
    }
    return(r0)
}


In [ ]:
#combine stay points based on dbscan
#the function will add a sid column
#for stayid not in a cluster, just set sid=-stayid
mergeStayPoint<-function(sp,eps=0.02,minp=2){
    cship2 <- as.matrix(sp[,list(lon,lat)])#get time series
    cl2 <- dbscan(cship2, eps = eps, minPts =minp)
    cship3=data.table(cbind(sp,sid=cl2$cluster))
    cship3=cship3[sid==0,sid:=(100000+stayid)]#pay attention on this line
    return(cship3) 
}

In [ ]:
#add stayid and sid to the original ship AIS records
setStayId<-function(s,sp){
    s=s[,stayid:=0]
    s=s[,sid:=0]
    n=nrow(sp)
    for(i in seq(1,n)){
        l=sp[i]
        s[pid<=l$endpid&pid>=l$startpid,stayid:=l$stayid]
        s[pid<=l$endpid&pid>=l$startpid,sid:=l$sid]
    }
    return(s)
    
}


In [ ]:
#set tripid to the original one
setTripId<-function(s,sp){
    
    n=nrow(sp)
    s=s[,tripid:=0]
    if(n>1){
        sp1=sp[1:(n-1),list(mmsi,startpid1=startpid,endpid1=endpid)]
        sp2=sp[2:n,list(startpid2=startpid,endpid2=endpid)]
        spln=cbind(sp1,sp2)
        for(i in seq(1,nrow(spln))){
            l=spln[i]
            s[pid>=l$endpid1&pid<=l$startpid2,tripid:=i]    
        }
    }
    return(s) 
}


In [ ]:
#add trips distance,duration, stayid,sid
#all of the functions are for a single ship
addTripStats<-function(trips,s){
    trips=trips[,dist:=0]
    trips=trips[,dur:=0]
    n=nrow(trips)
    for (i in (seq(1,n))){
        id=trips[i]$tripid
        trip=s[tripid==id]
        m=nrow(trip)
    
        if(m>1){
            trip1=trip[1:(m-1),list(mmsi,time1=time,lon1=lon,lat1=lat,sid1=sid)]
            trip2=trip[2:m,list(tripid2=tripid,time2=time,lon2=lon,lat2=lat,sid2=sid)] 
            tripln=cbind(trip1,trip2)
        
            tripln=tripln[,dist:=distance(lon1,lat1,lon2,lat2)]
            tripln=tripln[,dur:=(time2-time1)]
            totalDist=sum(tripln$dist)
            totalDur=sum(tripln$dur)
            trips[tripid==id,dist:=totalDist]
            trips[tripid==id,dur:=totalDur]
            trips[tripid==id,stayid1:=first(trip)$stayid]
            trips[tripid==id,stayid2:=last(trip)$stayid]
            trips[tripid==id,sid1:=first(trip)$sid]
            trips[tripid==id,sid2:=last(trip)$sid]
        }
    }
      
}

In [ ]:
#calculate trip statistics from original ais records 
#trips columns include:mmsi,tripid,N,dist,dur,stayid1,stayid2,sid1,sid2
getShipTripStats<-function(s){
    #individual ship
    dt=s[sog==0&status==5];
    sp=getStayPoint(dt,eps=3600*2,minp=5);sp=sp[stayid>0]
    sp=mergeStayPoint(sp,eps=0.02,minp=2)
    s=setStayId(s,sp)
    s=setTripId(s,sp)
    trips=s[tripid>0,.N,list(mmsi,tripid)];
    addTripStats(trips,s)
    return(trips)
}

# functions for an individual trip

In [ ]:
#get stay points based data with status!=5 and sog<5, mainly berth area
#also work for database with more then one mmsi
#sogLimit to set the speed limit of the select point
getTripStayPoint<-function(trip,soglimit=5,eps=0.002,minp=5){
    #add the first and last staypoint to the trip stay points
    dt=trip[sog<soglimit&status!=5]#very important
    if(nrow(dt)>0){
        cship2 <- as.matrix(dt[,list(lon,lat)])#get time series
        cl2 <- dbscan(cship2, eps = eps, minPts =minp)
        cship3=cbind(dt[,list(mmsi,time,sog,lon,lat,status,pid,stayid,sid,tripid)],tripstayid=cl2$cluster);
        tripStayPoint=cship3[tripstayid>0,list(startpid=first(.SD)$pid,endpid=last(.SD)$pid,duration=(last(.SD)$time-first(.SD)$time),lon=mean(.SD$lon),lat=mean(.SD$lat)),list(mmsi,tripid,tripstayid)] 
        tripStayPoint=tripStayPoint[,tripstayid:=(tripstayid+1)];
        firstStay=first(trip);firstStay=data.table(firstStay[,list(mmsi,tripid,tripstayid=1,startpid=pid,endpid=pid,duration=0,lon,lat)]);
        n=nrow(tripStayPoint)
        lastStay=last(trip);lastStay=data.table(lastStay[,list(mmsi,tripid,tripstayid=(n+2),startpid=pid,endpid=pid,duration=0,lon,lat)]);
        tripStayPoint=rbind(firstStay,tripStayPoint,lastStay);
    } 
    if(nrow(dt)==0){
        firstStay=first(trip);firstStay=data.table(firstStay[,list(mmsi,tripid,tripstayid=1,startpid=pid,endpid=pid,duration=0,lon,lat)]);
        lastStay=last(trip);lastStay=data.table(lastStay[,list(mmsi,tripid,tripstayid=2,startpid=pid,endpid=pid,duration=0,lon,lat)]);
        tripStayPoint=rbind(firstStay,lastStay);

    }
           
    return(tripStayPoint)
    

}


In [ ]:
#set tripstayid within an individual trip
# add a column called 'tripstayid '
#mainly for the anchor places 
setTripStayId<-function(trip,tripStayPoint){
    
    trip=trip[,tripstayid:=0]
    n=nrow(tripStayPoint);n
    for(i in seq(1,n)){
        l=tripStayPoint[i];
        trip[pid<=l$endpid&pid>=l$startpid,tripstayid:=l$tripstayid]
    }
    return(trip)
}


In [ ]:
#set subtripid for a trip
#add an column called subtripid

setTripSubTripId<-function(trip,tripStayPoint){
    n=nrow(tripStayPoint)
    trip=trip[,subtripid:=0]
    sp1=tripStayPoint[1:(n-1),list(mmsi,tripid,startpid1=startpid,endpid1=endpid)]
    sp2=tripStayPoint[2:n,list(startpid2=startpid,endpid2=endpid)]
    spln=cbind(sp1,sp2)
    for(i in seq(1,nrow(spln))){
        l=spln[i]
        trip[pid>=l$endpid1&pid<=l$startpid2,subtripid:=i]    
    } 
    return(trip)
}


In [ ]:
#add distance, duration,tripstayid1 and tripstayid2 to subtrips
addSubTripStats<-function(subtrips,trip){
    
    subtrips=subtrips[,dist:=0]
    subtrips=subtrips[,dur:=0]
    n=nrow(subtrips)
    for (i in (seq(1,n))){
        id=subtrips[i]$subtripid
        subtrip=trip[subtripid==id]
        m=nrow(subtrip)
  
        if(m>1){
            trip1=subtrip[1:(m-1),list(mmsi,time1=time,lon1=lon,lat1=lat)]
            trip2=subtrip[2:m,list(tripid2=tripid,time2=time,lon2=lon,lat2=lat)] 
            tripln=cbind(trip1,trip2)
        
            tripln=tripln[,dist:=distance(lon1,lat1,lon2,lat2)]
            tripln=tripln[,dur:=(time2-time1)]
            totalDist=sum(tripln$dist)
            totalDur=sum(tripln$dur)
            subtrips[subtripid==id,dist:=totalDist]
            subtrips[subtripid==id,dur:=totalDur]
            subtrips[subtripid==id,tripstayid1:=first(subtrip)$tripstayid]
            subtrips[subtripid==id,tripstayid2:=last(subtrip)$tripstayid]
      
        }
    }
    
    return(subtrips) 
}


# All process together

In [ ]:
#input: s[mmsi,time,sog,lon,lat,status] only for one ship
shipTraSegment<-function(s){
    
    #individual ship
    dt=s[sog==0&status==5];
    sp=getStayPoint(dt,eps=3600*2,minp=5);sp=sp[stayid>0]
    sp=mergeStayPoint(sp,eps=0.02,minp=2)
    s=setStayId(s,sp)
    s=setTripId(s,sp)
    trips=s[tripid>0,.N,list(mmsi,tripid)];
    addTripStats(trips,s)
    
    #individual trip------
    
    res=data.table(s[1],tripstayid=0,subtripid=0)[mmsi<0];
    
    n=nrow(trips)
    for(i in seq(1,n)){
        trip=s[tripid==trips[i]$tripid]
        setkey(trip,mmsi,time)
        #get trip stay point
        tripStayPoint=getTripStayPoint(trip,soglimit=5,eps=0.002,minp=5);
        #set trip stay id 
        trip=setTripStayId(trip,tripStayPoint)
        #set trip subtripid
        trip=setTripSubTripId(trip,tripStayPoint);
        res=rbind(res,trip)
        
    }
    #---------add subtripid and tripstayid to records with a tripid ==0, for example the stay points 
    temp=s[tripid==0]
    temp=temp[,tripstayid:=0]
    temp=temp[,subtripid:=0]
    res=rbind(res,temp)
    #--------------
    setkey(res,mmsi,time)
    return(res)
}

# main code

In [ ]:
#read data
s=fread('D:/Git/data/container/209207000.txt');dim(s);
setnames(s,c('mmsi','time','sog','lon','lat','status'));
s=cbind(s,pid=seq(1,nrow(s)));head(s,3);

In [ ]:
#time sample if needed
#tmp=s[,atime:=align.time(as.POSIXct(time,origin='1970-01-01'),60*5)];
#head(tmp,5);nrow(tmp[,.N,list(mmsi,atime)])

In [ ]:
#dt=s[time<(min(s$time)+30*24*3600)&sog==0&status==5];head(dt);plot(dt$time)
dt=s[sog==0&status==5];head(dt);plot(dt$time)

In [ ]:
#stayPoints
sp=getStayPoint(dt,eps=3600*2,minp=5);sp=sp[stayid>0];dim(sp);head(sp,3);

In [ ]:
zoomSize=9
    #temp=terminals;nrow(temp)
    temp=sp
    #t3[fcid%in%temp[,.N,fcid]$fcid]
    #temp=terminals[lon<0&lon>-180];nrow(temp)
    centerX=0.5*(max(temp$lon)+min(temp$lon))
    centerY=0.5*(max(temp$lat)+min(temp$lat))
    p<-ggmap(get_map(location=c(centerX,centerY),zoom=zoomSize,source='google',maptype = 'satellite'))
    p=p+geom_point(data=temp,aes(x=lon,y=lat,col=as.factor(stayid)),size=0.5,alpha=0.75)
    p=p+geom_text(data=temp,nudge_x = 0.0005,nudge_y = 0.0005,aes(x=lon,y=lat,label=stayid),color='black',size=2)
    p=p+labs(x="Longtitude",y="Latitude")+theme(legend.position='none')
    #p
    #hulls=temp[chull(temp[,list(lon,lat)]),]
    #p=p+geom_point(data = hulls, aes(x = lon, y = lat),size=1,col='yellow') 
    #p=p+geom_polygon(data = hulls, aes(x = lon, y = lat),fill='yellow',alpha = 0.4)
    plot(p)
    #ggsave(paste('D:/Git/data/portClusterPictures/','h','.png',sep=''))

In [ ]:
sp=mergeStayPoint(sp,eps=0.02,minp=2)
dim(sp);sp[,.N,sid];head(sp)

In [ ]:
s=setStayId(s,sp)
dim(s);head(s,3)

In [ ]:
s=setTripId(s,sp)
head(s[tripid==2]);dim(s)

In [ ]:
trips=s[tripid>0,.N,list(mmsi,tripid)];
addTripStats(trips,s)
head(trips);dim(trips);dim(s)

In [ ]:
setorder(trips,dist);head(trips,10);trips[sid1!=sid2,.N,list(sid1,sid2)]

In [ ]:
plot((log(trips$dist)));

# individual Trip test

In [ ]:
trip=s[tripid==33];dim(trip[sog==0&status!=5]);plot(trip$time,trip$sog)

In [ ]:
 zoomSize=14
    #temp=terminals;nrow(temp)
    temp=trip[sog<1&status!=5]
    #t3[fcid%in%temp[,.N,fcid]$fcid]
    #temp=terminals[lon<0&lon>-180];nrow(temp)
    centerX=0.5*(max(temp$lon)+min(temp$lon))
    centerY=0.5*(max(temp$lat)+min(temp$lat))
    p<-ggmap(get_map(location=c(centerX,centerY),zoom=zoomSize,source='google',maptype = 'satellite'))
    p=p+geom_point(data=temp,aes(x=lon,y=lat,col=as.factor(stayid)),size=0.5,alpha=0.75)
    p=p+geom_text(data=temp,nudge_x = 0.0005,nudge_y = 0.0005,aes(x=lon,y=lat,label=pid),color='black',size=2)
    p=p+labs(x="Longtitude",y="Latitude")+theme(legend.position='none')
    #p
    #hulls=temp[chull(temp[,list(lon,lat)]),]
    #p=p+geom_point(data = hulls, aes(x = lon, y = lat),size=1,col='yellow') 
    #p=p+geom_polygon(data = hulls, aes(x = lon, y = lat),fill='yellow',alpha = 0.4)
    plot(p)
    #ggsave(paste('D:/Git/data/portClusterPictures/','h','.png',sep=''))

In [ ]:
 zoomSize=8
    #temp=terminals;nrow(temp)
    temp=trip
    #t3[fcid%in%temp[,.N,fcid]$fcid]
    #temp=terminals[lon<0&lon>-180];nrow(temp)
    centerX=0.5*(max(temp$lon)+min(temp$lon))
    centerY=0.5*(max(temp$lat)+min(temp$lat))
    p<-ggmap(get_map(location=c(centerX,centerY),zoom=zoomSize,source='google',maptype = 'satellite'))
    p=p+geom_point(data=temp,aes(x=lon,y=lat,col=as.factor(stayid)),size=0.5,alpha=0.75)
    #p=p+geom_text(data=temp,nudge_x = 0.0005,nudge_y = 0.0005,aes(x=lon,y=lat,label=pid),color='black',size=2)
    p=p+labs(x="Longtitude",y="Latitude")+theme(legend.position='none')
    #p
    #hulls=temp[chull(temp[,list(lon,lat)]),]
    #p=p+geom_point(data = hulls, aes(x = lon, y = lat),size=1,col='yellow') 
    #p=p+geom_polygon(data = hulls, aes(x = lon, y = lat),fill='yellow',alpha = 0.4)
    plot(p)
    #ggsave(paste('D:/Git/data/portClusterPictures/','h','.png',sep=''))

In [ ]:
tripStayPoint=getTripStayPoint(trip,soglimit=5,eps=0.002,minp=5);head(tripStayPoint)
n=nrow(tripStayPoint);n

In [ ]:
trip=setTripStayId(trip,tripStayPoint)
trip[,.N,tripstayid];head(trip);dim(trip)

In [ ]:
trip=setTripSubTripId(trip,tripStayPoint);
head(trip);trip[,.N,subtripid]

In [ ]:
subtrips=trip[subtripid>0,.N,list(mmsi,tripid,subtripid)];
subtrips=addSubTripStats(subtrips,trip)
head(subtrips);dim(subtrips)

In [ ]:
res=getShipTripStats(s);dim(res);head(res)

In [ ]:
head(s);head(trip)

In [ ]:
res=shipTraSegment(s);dim(res);head(res,1);#do not include tripid==0

In [ ]:
#the total results
head(s,1)
head(sp,1)#stay points
head(trips,5)
head(trip)
head(tripStayPoint)#stay points
head(subtrips)

In [ ]:
#read data
s=fread('D:/Git/data/container/209506000.txt');dim(s);
setnames(s,c('mmsi','time','sog','lon','lat','status'));
s=cbind(s,pid=seq(1,nrow(s)));head(s,1);

In [ ]:
res=shipTraSegment(s);

In [ ]:
dim(res);head(res,5);